In [1]:
# Importación de liberías
import pandas as pd
from time import sleep

import bs4
import requests as rq
from bs4 import BeautifulSoup

import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By # Para poder usar el By

In [2]:
# Definición de la url base
url_base = 'https://comparador.cnmc.gob.es/'

In [3]:
#Inicialización driver
def init_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

In [4]:
#Cierre driver
def close_driver(driver):
    driver.close()

In [5]:
#Aceptación cookies
def accept_cookies(driver):
    driver.find_element(By.CLASS_NAME,'cookiesjsr-btn.important').click()

In [6]:
#Abrir lista de opciones
def selecct_sum(driver):
    driver.find_element(By.ID, "input-47").click()

In [7]:
#Selección de ofertas de electricidad en desplegable
def select_elect(driver):
    driver.find_element(By.ID, "list-item-59-0").click()

In [8]:
#Selección de ofertas de electricidad en desplegable
def select_gas(driver):
    driver.find_element(By.ID, "list-item-59-1").click()

In [9]:
#Selección de ofertas de electricidad en desplegable
def select_both(driver):
    driver.find_element(By.ID, "list-item-59-2").click()

In [10]:
#Selección de boton "Iniciar"
def iniciar(driver):
    driver.find_element(By.ID, "Iniciar").click()

In [11]:
#Escribe un código postal en formulario electricidad
def type_postal_code(driver):
    driver.find_element(By.NAME, "codigoPostal").click()
    driver.find_element(By.NAME, "codigoPostal").send_keys("08035")

In [12]:
#Continúa desde el formulario hasta la siguiente página
def continue_to_page(driver):
    driver.find_element(By.ID, "Continuar").click()
    sleep(1)
    driver.find_element(By.CLASS_NAME, "v-input--selection-controls__ripple").click()
    sleep(1)
    driver.find_element(By.XPATH, "(.//*[normalize-space(text()) and normalize-space(.)='He leído este aviso'])[1]/following::span[2]").click()
    return driver


In [13]:
#Navegación hasta página de ofertas de electricidad
def get_comparator_elect():
    driver = init_driver(url_base)
    accept_cookies(driver)
    selecct_sum(driver)
    select_elect(driver)
    iniciar(driver)
    sleep(3)
    type_postal_code(driver)
    driver = continue_to_page(driver)
    return driver

In [19]:
#Naveción hasta página de ofertas combinadas
def get_comparator_both():
    driver = init_driver(url_base)
    accept_cookies(driver)
    selecct_sum(driver)
    select_both(driver)
    iniciar(driver)
    sleep(3)
    type_postal_code(driver)
    continue_to_page(driver)
    return driver


In [24]:
#Extraccion de tabla de ofertas de electricidad
def get_elect_table(driver):

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all('table', attrs={'class': ''})

    for table in tables[1]:

        rows = table.find_all('tr', attrs={'class': ''})
        table_list = []

        for row in rows:

            data = row.find_all('td')
            row_list = []

            for elem in data:

                if data[0] == elem:

                    company = elem.find('img').attrs['alt']
                    row_list.append(company)
                elif data[1] == elem:

                    oferta = row.find('a').text.lstrip()
                    row_list.append(oferta)
                elif data[5] == elem:

                    servicios_adicionales = elem.find('div').string
                    row_list.append(servicios_adicionales)
                elif data[7] == elem:

                    eco = elem.find('i').attrs['class'][-1]

                    if eco == 'grey--text':
                        row_list.append(False)
                    elif eco == 'green--text':
                        row_list.append(True)
                elif data[8] == elem:
                    break
                else:
                    row_list.append(elem.string)

            table_list.append(row_list)

    df_elect = pd.DataFrame(table_list)
    df_elect.columns = ['Comercializadora', 'Oferta', 'Importe primera factura', 
    'Importe siguientes facturas', 'Validez', 'Servicios adicionales', 'Penalizacion', 'Verde']
    
    return df_elect

In [26]:
driver_elect = get_comparator_elect()
sleep(5)
df_elect = get_elect_table(driver_elect)
sleep(5)
close_driver(driver_elect)

In [ ]:
driver_elect = get_comparator_elect()
df_elect = get_elect_table(driver_elect)
sleep(5)
close_driver(driver_elect)

#get_comparator_gas()
#df_gas =get_gas_table()
#close_driver(driver)

driver_both = get_comparator_both()
#df_both = get_both_table(driver_both)
sleep(5)
close_driver(driver_both)